We begin by importing necessory libraries, viz. **Numpy** for matrix operations, **Sklearn** for accessign ready-made datasets and ML algorithms and **Matplotlib** for visualizing data.

In [ ]:
import numpy as np
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(0)
X, y = make_moons(200, noise=0.2)
plt.figure(figsize = (20,12))
plt.scatter(X[:,0], X[:,1], s=80, c=y, cmap=plt.cm.Spectral)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X, y)

In [ ]:
X1min, X1max = X[:,0].min()-0.5, X[:,0].max()+0.5
X2min, X2max = X[:,1].min()-0.5, X[:,1].max()+0.5
X1grid = np.arange(X1min, X1max, 0.01)
X2grid = np.arange(X2min, X2max, 0.01)
xx1, xx2 = np.meshgrid(X1grid, X2grid)
r1, r2 = xx1.flatten(), xx2.flatten()
r1, r2 = r1.reshape((len(r1), 1)), r2.reshape((len(r2), 1))
grid = np.hstack((r1,r2))
yhat = clf.predict(grid)
yy = yhat.reshape(xx1.shape)
plt.figure(figsize = (20,12))
plt.contourf(xx1, xx2, yy, cmap='Paired')
for y_ in range(2):
    ix = np.where(y == y_)
    plt.scatter(X[ix, 0], X[ix, 1], s=80, cmap='Paired')

In [ ]:
m = len(X)
ni = 2
no = 1
lr = 0.01
lmbda = 0.01 

In [ ]:
def predict(model, X):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    Z1 = W1.dot(X) + b1
    A1 = np.maximum(0, Z1)
    Z2 = W2.dot(A1) + b2
    A2 = 1/(1+np.exp(-1*Z2))
    return np.rint(A2)

In [ ]:
def cost(model, X, y):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    Z1 = W1.dot(X) + b1
    A1 = np.maximum(0, Z1)
    Z2 = W2.dot(A1) + b2
    A2 = 1/(1+np.exp(-1*Z2))
    logprobs = np.multiply(np.log(A2), y) + np.multiply((1 - y), np.log(1 - A2))
    cost = -np.sum(logprobs)
    cost += lmbda/2 * (np.sum(np.square(W1)) + np.sum(np.square(W2))) #L2 Regularization
    
    return cost/m

In [ ]:
def nnmodel(X, y, nh, ne=20000, print_loss=False):
    
    np.random.seed(0)
    W1 = np.random.randn(nh, ni)*0.01
    b1 = np.zeros((nh, 1))
    W2 = np.random.randn(no, nh)*0.01
    b2 = np.zeros((no, 1))
    
    model = {}
    
    for i in range(0, ne+1):
        
        Z1 = W1.dot(X) + b1
        A1 = np.maximum(0, Z1)
        Z2 = W2.dot(A1) + b2
        A2 = 1/(1+np.exp(-1*Z2))
        
        dZ2 = A2 - y
        dW2 = (dZ2).dot(A1.T)
        db2 = np.sum(dZ2)
        dZ1 = ((W2.T).dot(dZ2))*np.where(Z1>0, 1, 0)
        dW1 = (dZ1).dot(X.T)
        db1 = np.sum(dZ1)
        
        dW2 += lmbda*W2
        dW1 += lmbda*W1
        
        dW1 /= m
        dW2 /= m
        db1 /= m
        db2 /= m
        
        W1 += -lr * dW1
        b1 += -lr * db1
        W2 += -lr * dW2
        b2 += -lr * db2
        
        model = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        
        if print_loss and i % 1000 == 0:
            print('Epoch: {} \tLoss: {:.6f}'.format(i, cost(model, X, y)))
            
    return model

In [ ]:
ynn = y.reshape(1, m)
Xnn = X.T

In [ ]:
model = nnmodel(Xnn, ynn, nh=4, print_loss=True)

In [ ]:
X1min, X1max = X[:,0].min()-0.5, X[:,0].max()+0.5
X2min, X2max = X[:,1].min()-0.5, X[:,1].max()+0.5
X1grid = np.arange(X1min, X1max, 0.01)
X2grid = np.arange(X2min, X2max, 0.01)
xx1, xx2 = np.meshgrid(X1grid, X2grid)
r1, r2 = xx1.flatten(), xx2.flatten()
r1, r2 = r1.reshape((len(r1), 1)), r2.reshape((len(r2), 1))
grid = np.hstack((r1,r2))
yhat = predict(model, grid.T).reshape((len(r2)))
yy = yhat.reshape(xx1.shape)
print(yy)
plt.figure(figsize = (20,12))
plt.contourf(xx1, xx2, yy, cmap='Paired')
for y_ in range(2):
    ix = np.where(y == y_)
    plt.scatter(X[ix, 0], X[ix, 1], s=80, cmap='Paired')